<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Ressentis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classification de ressentis avec distilBERT**

L'objectif est de créer un modèle qui prend en entrée des commentaires (en Anglais) et attribue à chacun un ressenti positif ou négatif.  
Le modèle est composé de deux parties :  
* DistilBERT va encoder le commentaire et en extraire des informations qui seront passées ensuite au réseau de neurones.  
* Le modèle suivant est un réseau de neurones qui sera créé avec Keras.  

<img src="https://raw.githubusercontent.com/AlexandreBourrieau/ML-F1/master/Carnets%20Jupyter/Images/StructureBERT.png" />  
  
  Les données qui s'échangent entre les deux modèles sont des vecteurs de dimension 768. Ces vecteurs sont l'équivalent de l'application d'un algorithme de prolongation lexicale sur les mots qui composent le commentaire.

# **Installation et importations des librairies**


In [1]:
!pip install transformers --quiet

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 19.2MB/s 
     |████████████████████████████████| 890kB 34.2MB/s 
     |████████████████████████████████| 1.1MB 35.3MB/s 


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, Input, Dropout, Lambda
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

from transformers import TFDistilBertModel
from transformers import AutoTokenizer

import matplotlib.pyplot as plt

import random

# **Importation des données**

On utilise la librairie pandas pour lire les données depuis le fichier csv disponible sur le site de [standford](https://nlp.stanford.edu/sentiment/index.html) qui contient des commentaires sur des films, chacun d'eux avec une note positive (1) ou négative (0).

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/AlexandreBourrieau/ML-F1/master/Carnets%20Jupyter/Donn%C3%A9es/train.csv', delimiter='\t', header=None)

Affiche quelques informations :

In [4]:
print(df[0:10])
print("Total des données : ", str(len(df)))
print("Nombre d'avis positifs et négatifs : ",df[1].value_counts())

                                                   0  1
0  a stirring , funny and finally transporting re...  1
1  apparently reassembled from the cutting room f...  0
2  they presume their audience wo n't sit still f...  0
3  this is a visually stunning rumination on love...  1
4  jonathan parker 's bartleby should have been t...  1
5  campanella gets the tone just right funny in t...  1
6  a fan film that for the uninitiated plays bett...  0
7  b art and berling are both superb , while hupp...  1
8  a little less extreme than in the past , with ...  0
9                       the film is strictly routine  0
Total des données :  6920
Nombre d'avis positifs et négatifs :  1    3610
0    3310
Name: 1, dtype: int64


# **Préparation des données**

# Tokénisation  
La première étape est de tokéniser les commentaires : les mots sont décomposés en index numériques au format BERT.  

<img src="https://github.com/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Images/TokenizeBERT.png?raw=true"/>

Après tokénisation, on obtient une liste de séquences et chaque séquence représente une liste d'index. On souhaite que BERT analyse toutes les séquences en une seule fois (ce qui est plus rapide). Il faut donc que toutes les séquences aient la même taille. On va donc ajouter du bourrage pour égaliser la longueur des séquences. Cela est indiqué avec le paramètre `padding='True'`.

In [40]:
MAX_SEQUENCE_LENGTH = 500

# Chargement des commentaires et des ressentis
commentaires = df[0]                                   # Récupère tous les commentaires dans une dataframe pandas
ressentis = df[1]                                      # Récupère tous les ressentis dans une liste python
labels = np.asarray(ressentis)                         # Créé un tableau de type numpy avec les ressentis

x_entrainement, x_test, y_entrainement, y_test = train_test_split(commentaires, labels, test_size=0.25)

In [42]:
# Instanciation du tokeniseur
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Préparation des données d'entrainement
output_tokenizer_entrainement = x_entrainement.apply((lambda x: tokenizer.encode(x,max_length=MAX_SEQUENCE_LENGTH, padding='longest', truncation=True, return_tensors='tf',add_special_tokens=True)))

# Préparation des données de tests
output_tokenizer_tests = x_test.apply((lambda x: tokenizer.encode(x,max_length=MAX_SEQUENCE_LENGTH, padding='longest', truncation=True, return_tensors='tf',add_special_tokens=True)))

Regardons un peu comment sont formatées les données en sortie du tokéniseur :

In [48]:
print("Commentaire original :", x_entrainement[1])
print("Résultat de la tokénisation: ", output_tokenizer_entrainement[1])

Commentaire original : apparently reassembled from the cutting room floor of any given daytime soap
Résultat de la tokénisation:  tf.Tensor(
[[  101  4593  2128 27241 23931  2013  1996  6276  2282  2723  1997  2151
   2445 12217  7815   102]], shape=(1, 16), dtype=int32)


Regardons les 5 premiers résutats de la tokénisation : On peut identifier les mot-clés **[CLS]** (valeur : 101) et **[SEP]** (valeur : 102)

In [47]:
for i in range (0,5):
  print(output_tokenizer_entrainement[i])

tf.Tensor(
[[  101  1037 18385  1010  6057  1998  2633 18276  2128 16603  1997  5053
   1998  1996  6841  1998  5687  5469  3152   102]], shape=(1, 20), dtype=int32)
tf.Tensor(
[[  101  4593  2128 27241 23931  2013  1996  6276  2282  2723  1997  2151
   2445 12217  7815   102]], shape=(1, 16), dtype=int32)


KeyError: ignored

# Bourrage
Après tokénisation, `output_tokenizer` est une liste de séquences, chaque séquence représente une liste d'index. On souhaite que BERT analyse toutes les séquences en une seule fois (ce qui est plus rapide). Il faut donc que toutes les séquences aient la même taille. On va donc ajouter du bourrage pour égaliser la longueur des séquences.

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > MAX_SEQUENCE_LENGTH:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# **Définition du modèle**